In [37]:
import numpy as np 
import pandas as pd 
import math
from collections import Counter
from nltk import word_tokenize

In [38]:
def tf(question, word):
    if word not in question:
        return 0
    count = dict(Counter(question))
    q_len = len(question)
    return float(count[word]) / float(q_len)

In [39]:
def n_containing(qlist, word):
    return float(qlist[word])

def idf(qlist, word):
    return math.log(float(len(qlist.keys())) / (1.0 + n_containing(qlist, word)))

In [40]:
def tfidf(question, qlist, word):
    return tf(question, word) * idf(qlist, word)

In [41]:
def cosine(v1, v2):
    v1 = np.array(v1)
    v2 = np.array(v2)
    return np.dot(v1, v2) / (np.sqrt(np.sum(v1**2)) * np.sqrt(np.sum(v2**2)))

In [42]:
train = pd.read_csv('final_merged_data.csv')

train_qs = train[['Leading Comment','Category', 'Forum']]

qlist = []
count = 0
for row in train_qs.itertuples():
    try:
        if len(str(row[2])) > 10:
            q1 = word_tokenize(row[2].lower())
        if len(str(row[3])) > 10:
            q2 = word_tokenize(row[3].lower())
        qlist += q1 + q2
        count+=1
        if count%100000 == 0:
            print('At'+str(count))
    except TypeError:
        pass

print (len(qlist))
qlist = dict(Counter(qlist))
import json
with open('qlist.json', 'w') as f:
    f.write(json.dumps(qlist, indent=2))
print('All commments added to list')

61426
All commments added to list


In [43]:
with open('commentsimilarity.csv', 'a') as f:
    f.write('id,is_duplicate\n')
for row in train_qs.itertuples():
    if len(str(row[2])) > 10 and len(str(row[3])) > 10:
        wordvec1 = word_tokenize(row[2].lower())
        wordvec2 = word_tokenize(row[3].lower())
        words = wordvec1 + wordvec2
        words = list(set([word for word in words if word != '?']))

        # print words

        vec1 = []
        vec2 = []
        for word in words:
            vec1.append(tfidf(wordvec1, qlist, word))
            vec2.append(tfidf(wordvec2, qlist, word))

        with open('commentsimilarity.csv', 'a') as f:
            f.write(str(row[1]) + "," + str(cosine(vec1, vec2)) + '\n')
    else:
        with open('commentsimilarity.csv', 'a') as f:
            f.write(str(row[1]) + "," + '0' + '\n')

In [44]:
print (str(row[1]) + "," + str(cosine(vec1, vec2)))

Have questions about Store & Website Management? This is the category to use. Please be sure to select the most appropriate sub-category for your questions.,0.0
